In [94]:
with open('input.txt') as f:
    lines = f.readlines()

In [95]:
def is_valid(s):
    if '@#' in s or '#@' in s:
        return False
    first_at = s.rfind('@')
    for i in range(first_at):
        if s[i] == '#':
            return False
    else:
        return True

In [96]:
def place_substring(s, length):
    pos = []
    for i in range(len(s) - length+1):
        str_overwritten = s[i:i+length]
        if '.' in str_overwritten or '@' in str_overwritten:
            continue
        cur_str = s[:i] + length * '@' + s[i+length:]
        if is_valid(cur_str):
            # Replace the chars with '.'
            cur_str = cur_str.replace('@?', '@.').replace('?@', '.@')
            # Replace all ? left of the first @ with .
            first_new_at = i
            for j in range(first_new_at):
                if cur_str[j] == '?':
                    cur_str = cur_str[:j] + '.' + cur_str[j+1:]
            pos.append(cur_str)
    return pos

In [97]:
def get_all_pos(s, lengths, total_l):
    if s.count('#') == 0 and s.count('@') == total_l:
        return 1
    if len(lengths) == 0:
        return 0
    else:
        lengths_copy = lengths.copy()
        next_poss = place_substring(s, lengths_copy.pop(0))
        total = 0
        for pos in next_poss:
            # Check first if it could even be a solution:
            if pos.count('#') > total_l:
                continue
            else:
                total += get_all_pos(pos, lengths_copy, total_l)
        return total
        


In [98]:
examples = []
for line in lines:
    example = line.strip().split(' ')
    example[0] = "?".join([example[0] for _ in range(5)])
    example[1] = ",".join([example[1] for _ in range(5)])             
    example[1] = [int(x) for x in example[1].split(',')]
    examples.append(example)

In [99]:
#?#???????#?.? 3,1,2,2

In [100]:
def process_example(example):
    return get_all_pos(example[0], example[1], sum(example[1]))

In [101]:
examples[0]

['#?#???????#?.??#?#???????#?.??#?#???????#?.??#?#???????#?.??#?#???????#?.?',
 [3, 1, 2, 2, 3, 1, 2, 2, 3, 1, 2, 2, 3, 1, 2, 2, 3, 1, 2, 2]]

In [102]:
process_example(examples[0])

58564

In [83]:
from tqdm import tqdm
from multiprocessing import Pool
import os



# Assuming 'examples' is a list of tuples
with Pool(os.cpu_count()) as pool:
    results = list(tqdm(pool.imap(process_example, examples), total=len(examples)))

total = sum(results)


100%|██████████| 1000/1000 [00:00<00:00, 8783.22it/s]


In [84]:
total

7007